In [10]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   


# Load data

In [11]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

# all edges ON (example); add node/edge 0s as needed
states = {eid: 1 for eid in edges.keys()}

k_val, sys_st, _ = fbg.eval_global_conn_k(states, G_base)
print("k =", k_val, "System state =", sys_st)

k = 2 System state = 2


In [12]:
s_fun = lambda comps_st: fbg.eval_global_conn_k(comps_st, G_base)
row_names = list(edges.keys())
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [13]:
TSUMPATH = Path("tsum_res") 
sys_surv_st = 2 # either 1 or 2

rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

C:\Users\jb622s\AppData\Local\Temp\ipykernel_8664\672774486.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_

# Calculate system probabilities

## Marginal probability

In [15]:
import importlib
importlib.reload(tsum)

<module 'tsum.tsum' from 'c:\\Users\\jb622s\\git\\tsum\\tsum\\tsum.py'>

In [16]:
pr_cond = tsum.get_comp_cond_sys_prob(
    rules_mat_surv,
    rules_mat_fail,
    probs,
    comps_st_cond = {},
    row_names = row_names,
    s_fun = s_fun,
    sys_surv_st=sys_surv_st
)
print(f"P(sys >= {sys_surv_st}) = {pr_cond['survival']:.3f}")
print(f"P(sys <= {sys_surv_st-1}) = {pr_cond['failure']:.3f}\n")

P(sys >= 2) = 0.168
P(sys <= 1) = 0.832



## Conditional probability given "one" component's survival

In [17]:
for x in row_names[:-1]:
    print(f"Eval P(sys | {x}=1)")
    pr_cond = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun,
        sys_surv_st=sys_surv_st
    )
    print(f"P(sys >= {sys_surv_st} | {x}=1) = {pr_cond['survival']:.3f}")
    print(f"P(sys <= {sys_surv_st-1} | {x}=1) = {pr_cond['failure']:.3f}\n")

Eval P(sys | e01=1)
P(sys >= 2 | e01=1) = 0.168
P(sys <= 1 | e01=1) = 0.832

Eval P(sys | e02=1)
P(sys >= 2 | e02=1) = 0.210
P(sys <= 1 | e02=1) = 0.790

Eval P(sys | e03=1)
P(sys >= 2 | e03=1) = 0.210
P(sys <= 1 | e03=1) = 0.790

Eval P(sys | e04=1)
P(sys >= 2 | e04=1) = 0.210
P(sys <= 1 | e04=1) = 0.790

Eval P(sys | e05=1)
P(sys >= 2 | e05=1) = 0.210
P(sys <= 1 | e05=1) = 0.790

Eval P(sys | e06=1)
P(sys >= 2 | e06=1) = 0.168
P(sys <= 1 | e06=1) = 0.832

Eval P(sys | e07=1)
P(sys >= 2 | e07=1) = 0.210
P(sys <= 1 | e07=1) = 0.790

Eval P(sys | e08=1)
P(sys >= 2 | e08=1) = 0.210
P(sys <= 1 | e08=1) = 0.790

Eval P(sys | e09=1)
P(sys >= 2 | e09=1) = 0.210
P(sys <= 1 | e09=1) = 0.790

Eval P(sys | e10=1)
P(sys >= 2 | e10=1) = 0.168
P(sys <= 1 | e10=1) = 0.832



## Conditional probability given "two" component state

In [ ]:
for i, x in enumerate(row_names[:-1]):
    for j, y in enumerate(row_names[i+1:-1]):
        
        print(f"Eval P(sys | {x}=1, {y}=1)")
        pr_cond = tsum.get_comp_cond_sys_prob(
            rules_mat_surv,
            rules_mat_fail,
            probs,
            comps_st_cond = {x: 1, y: 1},
            row_names = row_names,
            s_fun = s_fun,
            sys_surv_st=sys_surv_st
        )
        print(f"P(sys >= {sys_surv_st} | {x}=1, {y}=1) = {pr_cond['survival']:.3f}")
        print(f"P(sys <= {sys_surv_st-1} | {x}=1, {y}=1) = {pr_cond['failure']:.3f}\n")
   

Eval P(sys | e01=1, e02=1)
P(sys=1 | e01=1, e02=1) = 0.209
P(sys=0 | e01=1, e02=1) = 0.791

Eval P(sys | e01=1, e03=1)
P(sys=1 | e01=1, e03=1) = 0.210
P(sys=0 | e01=1, e03=1) = 0.790

Eval P(sys | e01=1, e04=1)
P(sys=1 | e01=1, e04=1) = 0.209
P(sys=0 | e01=1, e04=1) = 0.791

Eval P(sys | e01=1, e05=1)
P(sys=1 | e01=1, e05=1) = 0.210
P(sys=0 | e01=1, e05=1) = 0.790

Eval P(sys | e01=1, e06=1)
P(sys=1 | e01=1, e06=1) = 0.167
P(sys=0 | e01=1, e06=1) = 0.833

Eval P(sys | e01=1, e07=1)
P(sys=1 | e01=1, e07=1) = 0.210
P(sys=0 | e01=1, e07=1) = 0.790

Eval P(sys | e01=1, e08=1)
P(sys=1 | e01=1, e08=1) = 0.210
P(sys=0 | e01=1, e08=1) = 0.790

Eval P(sys | e01=1, e09=1)
P(sys=1 | e01=1, e09=1) = 0.210
P(sys=0 | e01=1, e09=1) = 0.790

Eval P(sys | e01=1, e10=1)
P(sys=1 | e01=1, e10=1) = 0.168
P(sys=0 | e01=1, e10=1) = 0.832

Eval P(sys | e02=1, e03=1)
P(sys=1 | e02=1, e03=1) = 0.263
P(sys=0 | e02=1, e03=1) = 0.737

Eval P(sys | e02=1, e04=1)
P(sys=1 | e02=1, e04=1) = 0.263
P(sys=0 | e02=1, e04=